In [3]:
import kgbench as kg

data = kg.load('dmg777k') # Load with numpy arrays, and train/validation split

# data = kg.load('dmg777k', torch=True) # Load with pytorch arrays

# data = kg.load('dmg777k', final=True) # Load with numpy arrays and train/test split


loaded data dmg777k (38.97s).


In [5]:
print(data.num_classes)
data.datatypes()

5


['iri',
 'none',
 '@es',
 '@fy',
 '@nl',
 '@nl-nl',
 '@pt',
 '@ru',
 'http://kgbench.info/dt#base64Image',
 'http://www.opengis.net/ont/geosparql#wktLiteral',
 'http://www.w3.org/2001/XMLSchema#anyURI',
 'http://www.w3.org/2001/XMLSchema#boolean',
 'http://www.w3.org/2001/XMLSchema#gYear',
 'http://www.w3.org/2001/XMLSchema#nonNegativeInteger',
 'http://www.w3.org/2001/XMLSchema#positiveInteger']

In [ ]:
print(data.)

In [3]:
data.withheld

array([[288312,      3],
       [287116,      0],
       [287208,      1],
       ...,
       [287084,      3],
       [286606,      1],
       [288132,      1]], dtype=int32)

In [4]:
def create_uri_classes_lists(data, data_set):
    list_of_uris = []
    list_of_classes=[]
    for data_instance in data_set:
    #     print(data_instance)
        list_of_uris.append(data.i2e[data_instance[0]][0])
        list_of_classes.append(data_instance[1])
    return list_of_uris, list_of_classes

In [5]:
list_of_uris_train, list_of_classes_train = create_uri_classes_lists(data, data.training)

In [6]:
list_of_uris, list_of_classes = create_uri_classes_lists(data, data.withheld)

In [7]:
def show_lists(list1, list2):
    for i, (uri, y) in enumerate(zip(list1, list2)):
        print(uri, y)
        if i > 1:
            break
    print(len(list1), len(list2))
    
show_lists(list_of_uris, list_of_classes)
show_lists(list_of_uris_train, list_of_classes_train)


https://data.labs.pdok.nl/rce/id/monument/8800 3
https://data.labs.pdok.nl/rce/id/monument/529502 0
https://data.labs.pdok.nl/rce/id/monument/532 1
1001 1001
https://data.labs.pdok.nl/rce/id/monument/36903 1
https://data.labs.pdok.nl/rce/id/monument/24531 4
https://data.labs.pdok.nl/rce/id/monument/27163 1
5394 5394


In [8]:
data.i2e[286290]

('https://data.labs.pdok.nl/rce/id/monument/519146', 'iri')

In [11]:
import time

In [12]:
import rdflib

start_time = time.time()
g = rdflib.Graph()
g.parse("data/dmg777k_stripped_without_thumbs_geo-.nt")
print("--- %s seconds ---" % (time.time() - start_time))

Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#date, Converter=<function parse_date at 0x7f7ac9f3e820>
Traceback (most recent call last):
  File "/Users/roderickvanderweerdt/opt/anaconda3/envs/gcn_env/lib/python3.9/site-packages/rdflib/term.py", line 2084, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
  File "/Users/roderickvanderweerdt/opt/anaconda3/envs/gcn_env/lib/python3.9/site-packages/isodate/isodates.py", line 201, in parse_date
    return date(sign * int(groups['year']),
ValueError: month must be in 1..12
Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#date, Converter=<function parse_date at 0x7f7ac9f3e820>
Traceback (most recent call last):
  File "/Users/roderickvanderweerdt/opt/anaconda3/envs/gcn_env/lib/python3.9/site-packages/rdflib/term.py", line 2084, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
  File "/Users/roderic

Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#date, Converter=<function parse_date at 0x7f7ac9f3e820>
Traceback (most recent call last):
  File "/Users/roderickvanderweerdt/opt/anaconda3/envs/gcn_env/lib/python3.9/site-packages/rdflib/term.py", line 2084, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
  File "/Users/roderickvanderweerdt/opt/anaconda3/envs/gcn_env/lib/python3.9/site-packages/isodate/isodates.py", line 201, in parse_date
    return date(sign * int(groups['year']),
ValueError: year 0 is out of range
Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#date, Converter=<function parse_date at 0x7f7ac9f3e820>
Traceback (most recent call last):
  File "/Users/roderickvanderweerdt/opt/anaconda3/envs/gcn_env/lib/python3.9/site-packages/rdflib/term.py", line 2084, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
  File "/Users/roderic

--- 144.27369499206543 seconds ---


In [26]:
q = """
SELECT DISTINCT ?monument_uri ?val
WHERE {
    ?monument_uri <https://data.labs.pdok.nl/rce/def/bouwjaar> ?val .
}
"""

q_pop = """
SELECT DISTINCT ?monument_uri ?val
WHERE {
    ?monument_uri <http://dbpedia.org/ontology/city> ?city .
    ?city <http://www.geonames.org/ontology#population> ?val .
}
"""

q_code = """
SELECT DISTINCT ?monument_uri ?val
WHERE {
    ?monument_uri <http://dbpedia.org/ontology/codeNationalMonument> ?val .
}
"""

#monument_values = {year: <int>, population: <int>, codeNationalMonument: <int>}

def add_monument_value(monument_val_dict, value_name, graph, query):
    qres = graph.query(query)
    print("query returned", len(qres), "results.")
    new_dict = 0
    existing_dict = 0
    doubles = 0
    for row in qres:
        monument_str = str(row.monument_uri)
        try:
            try:
#                 monument_val_dict[monument_str][value_name] = int((int(monument_val_dict[monument_str][value_name]) + int(row.val))/2)
                if monument_val_dict[monument_str][value_name] > int(row.val):
                    monument_val_dict[monument_str][value_name] = int(row.val)
                doubles += 1
            except:  
                monument_val_dict[monument_str][value_name] = int(row.val)
                existing_dict += 1
        except:
            new_dict += 1
            monument_val_dict[monument_str] = {}
            monument_val_dict[monument_str][value_name] = int(row.val)
    print("values for new monuments:", new_dict, "\nvalues for previous monuments:", existing_dict, "\nmultiple values for same monuments:", doubles)
    print("monument_val_dict now contains values for", len(monument_val_dict), "monuments.\n")
    return monument_val_dict

value_list = ["year", "population", "codeNationalMonument"]
query_list = [q, q_pop, q_code]
monument_val_dict = {}

start_time = time.time()
for value_name, query in zip(value_list, query_list):
    monument_val_dict = add_monument_value(monument_val_dict, value_name, g, query)
print("--- %s seconds ---" % (time.time() - start_time))

query returned 1800 results.
values for new monuments: 1430 
values for previous monuments: 0 
multiple values for same monuments: 370
monument_val_dict now contains values for 1430 monuments.

query returned 5179 results.
values for new monuments: 4325 
values for previous monuments: 854 
multiple values for same monuments: 0
monument_val_dict now contains values for 5755 monuments.

query returned 8396 results.
values for new monuments: 2641 
values for previous monuments: 5755 
multiple values for same monuments: 0
monument_val_dict now contains values for 8396 monuments.

--- 0.9246320724487305 seconds ---


In [14]:
def shuffle_dataset(dataset_fn, new_dataset_fn='0'):
    if new_dataset_fn == '0':
        new_dataset_fn = dataset_fn

    df = pd.read_csv(dataset_fn, sep=",")
    df = df.sample(frac=1)
    df.to_csv(new_dataset_fn, sep=",", index=False)
    

def print_values(val_list, val_dict):
    values = ""
    for val in val_list:
        try:
            values += str(val_dict[val])+','
        except:
            values += ','
    return values[:-1] #remove last ','
        
def write_entities_dmg(list_of_entities, list_of_y, val_list, entity_val_dict, new_fn):
    un_found = [0,0]
    with open(new_fn, 'w') as f:
        f.write("entity,y,"+','.join(val_list)+'\n')
        for entity, y in zip(list_of_entities, list_of_y):
            try:
                f.write(entity+','+str(y)+','+print_values(val_list, entity_val_dict[entity])+'\n')
                un_found[0] += 1
            except:
                f.write(entity+','+str(y)+','+print_values(val_list, {})+'\n')
#                 print(entity+','+str(y)+','+print_values(val_list, {}))
#                 print("no values found for entity:", entity)
                un_found[1] += 1
    print("found values for", un_found[0], "entities")
    print("couldn't find values for", un_found[1], "entities")

In [20]:
write_entities_dmg(list_of_uris_train, 
                   list_of_classes_train, 
                   value_list, 
                   monument_val_dict, 
                   "data/entities_dgm777k.csv")

write_entities_dmg(list_of_uris, 
                   list_of_classes, 
                   value_list, 
                   monument_val_dict, 
                   "data/entities_dgm777k_VALIDATION.csv")


found values for 5394 entities
couldn't find values for 0 entities
found values for 1001 entities
couldn't find values for 0 entities


In [21]:
import pandas as pd

In [22]:
shuffle_dataset("data/entities_dgm777k.csv")
shuffle_dataset("data/entities_dgm777k_VALIDATION.csv")

In [23]:
def combine_files(update_fn, train_fn, combined_fn):
    update_entities_df = pd.read_csv(update_fn, sep=",")
    train_entities_df = pd.read_csv(train_fn, sep=",")
    
    combined_entities_df = pd.concat([train_entities_df, update_entities_df])
    combined_entities_df.to_csv(combined_fn, index=False)
    
def subtract_files(complete_fn, subtract_fn, result_fn, entity_column="entity"):
    complete_df = pd.read_csv(complete_fn)
    subtract_df = pd.read_csv(subtract_fn)
    complete_df.drop(complete_df[complete_df[entity_column].isin(subtract_df[entity_column])].index, inplace = True)
    complete_df.to_csv(result_fn, index=False)

In [24]:
update_filename = "data/entities_dgm777k_VALIDATION.csv"
train_filename = "data/entities_dgm777k.csv"
combined_filename = "data/entities_dmg777k_COMBINED.csv"

combine_files(update_filename, train_filename, combined_filename)

create embeddings

In [25]:
# emb_file = "data/entities_dmg777k_COMBINED_w_emb.csv"
emb_file = "data/dmg777k_stripped_without_thumbs_geo_w_emb.csv"

update_w_emb_fn = "data/entities_dmg777k_VALIDATION_w_emb.csv"
train_w_emb_fn = "data/entities_dmg777k_TRAIN_w_emb.csv"

subtract_files(emb_file, train_filename, update_w_emb_fn)
subtract_files(emb_file, update_filename, train_w_emb_fn)

In [2]:
from classifier_mlp_AM import perform_prediction

Create the model

In [3]:
final_results = perform_prediction(dataset_fn=train_w_emb_fn, show_all=True, save_model=True)

Shape of X [N, C, H, W]:  torch.Size([8, 100])
Shape of y:  torch.Size([8]) torch.int64
Using cpu device
creating a classifier for 5 classes.
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=100, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=5, bias=True)
  )
)
Epoch 1
-------------------------------
loss: 1.595851  [    0/ 5394]
loss: 1.298305  [  800/ 5394]
loss: 0.773514  [ 1600/ 5394]
loss: 0.889103  [ 2400/ 5394]
loss: 0.555529  [ 3200/ 5394]
loss: 0.951278  [ 4000/ 5394]
loss: 0.835354  [ 4800/ 5394]
Train Accuracy: 65.7%
Test Accuracy: 63.3%
Epoch 2
-------------------------------
loss: 0.886922  [    0/ 5394]
loss: 0.893776  [  800/ 5394]
loss: 0.416718  [ 1600/ 5394]
loss: 0.623925  [ 2400/ 5394]
loss: 0.637962  [ 3200/ 5394]
loss: 0.793822  [ 4000/ 5394]
loss: 0.610516  [ 4800/ 539

loss: 0.012342  [ 2400/ 5394]
loss: 0.023688  [ 3200/ 5394]
loss: 0.014811  [ 4000/ 5394]
loss: 0.023047  [ 4800/ 5394]
Train Accuracy: 100.0%
Test Accuracy: 59.1%
Epoch 28
-------------------------------
loss: 0.004647  [    0/ 5394]
loss: 0.016767  [  800/ 5394]
loss: 0.002500  [ 1600/ 5394]
loss: 0.010641  [ 2400/ 5394]
loss: 0.020723  [ 3200/ 5394]
loss: 0.012993  [ 4000/ 5394]
loss: 0.020802  [ 4800/ 5394]
Train Accuracy: 100.0%
Test Accuracy: 59.2%
Epoch 29
-------------------------------
loss: 0.004428  [    0/ 5394]
loss: 0.015072  [  800/ 5394]
loss: 0.002135  [ 1600/ 5394]
loss: 0.009017  [ 2400/ 5394]
loss: 0.018448  [ 3200/ 5394]
loss: 0.011481  [ 4000/ 5394]
loss: 0.018433  [ 4800/ 5394]
Train Accuracy: 100.0%
Test Accuracy: 59.4%
Epoch 30
-------------------------------
loss: 0.004262  [    0/ 5394]
loss: 0.013656  [  800/ 5394]
loss: 0.001890  [ 1600/ 5394]
loss: 0.007985  [ 2400/ 5394]
loss: 0.016809  [ 3200/ 5394]
loss: 0.010548  [ 4000/ 5394]
loss: 0.016824  [ 4800/ 5